    Author: Roman Makarov
    e-mail: mcronomus@gmail.com

# Task Description

You have to move several cargos across the gridworld into a common desirable rectangle area. There are several cargos, you can move each of them either horizontally or vertically by one cell up or down. Size of the overall world may vary, as well as placement of the cargos and desirable area. The game ends when all the cargos are in the desirable area and do not overlap.

Reward function:
* for each cell of cargo placed in desirable area in the end of the turn, reward is $+1$
* for each cell where cargos overlap in the end of the turn, reward is $-1$

`{infile}.txt` file with the field description. Elements of the field are separated by space. For example:

```
0 0 2 2 0
0 r r r 0
0 r r r 1
0 r r r 1
0 0 0 0 1
```

* `0` - blank space, we may move objects here
* `r` - desirable area, we should move objects here
* `1`, `2`, ... - actual object shapes, does not change, moved as a solid object

In your output file, you have to record lines in the following manner: `{id} {D/U/R/L}`. For the given example, possible sequence of steps is:
```
2 D
1 L
1 U
2 L
```

Here the rewards are:
1. +2 for 2 cells of `2`
2. +4 for 2 cells of `2` and 2 cells of `1`
3. +3 (= +4 - 1) for 2 cells of `2` and 2 cells of `1` and -1 overlapping cell
4. **The end**

#Solution

##Data preparation

In [ ]:
import itertools
import numpy as np
from typing import Dict, Tuple

In [ ]:
def read_world(file_):
    world_ = []
    for line in open(file_).readlines():
        world_.append([_x for _x in line.split()])

    # Find the boundaries of the desirable area
    top_left = (-1, -1)
    bottom_right = (-1, -1)
    # And coordinates of the cargos
    cargos_coordinates = dict()

    for i_ in range(len(world_)):
        for j_ in range(len(world_[i_])):
            if world_[i_][j_] == 'r':
                if top_left[0] == -1:
                    top_left = [i_, j_]
                bottom_right = [i_, j_]
            elif world_[i_][j_] != '0':
                index_ = int(world_[i_][j_])
                if index_ not in cargos_coordinates:
                    cargos_coordinates[index_] = list()
                cargos_coordinates[index_].append([i_, j_])

    return world_, cargos_coordinates, top_left, bottom_right

In [ ]:
def prepare_data(path_to_infile):
    # reading and creating the world
    world_shape, cargos_coordinates, top_left, bottom_right = read_world(path_to_infile)

    cargos_corners = dict()
    for cargo in cargos_coordinates:
        top, bottom, left, right = 0, 0, 0, 0
        for i in range(len(cargos_coordinates[cargo])):
            if cargos_coordinates[cargo][top][0] > cargos_coordinates[cargo][i][0]:
                top = i
            if cargos_coordinates[cargo][bottom][0] < cargos_coordinates[cargo][i][0]:
                bottom = i
            if cargos_coordinates[cargo][left][1] > cargos_coordinates[cargo][i][1]:
                left = i
            if cargos_coordinates[cargo][right][1] > cargos_coordinates[cargo][i][1]:
                right = i

        cargos_corners[cargo] = [cargos_coordinates[cargo][top][0], cargos_coordinates[cargo][right][1],
                                 cargos_coordinates[cargo][bottom][0], cargos_coordinates[cargo][left][1]]

    return world_shape, cargos_coordinates, cargos_corners, top_left, bottom_right

In [ ]:
def print_world(world_shape, cargos_coordinates, top_left, bottom_right):
    printed_world = [[0 for y in range(len(x))] for x in world_shape]

    for i in range(top_left[0], bottom_right[0] + 1):
        for j in range(top_left[1], bottom_right[1] + 1):
            printed_world[i][j] = 'r'

    for Cargo in cargos_coordinates:
        for coordinates in cargos_coordinates[Cargo]:
            printed_world[coordinates[0]][coordinates[1]] = Cargo

    for line in printed_world:
        print(*line)

###A function for generating random samples for training

In [ ]:
import random

def generate_random_map(file_name, min_n=4, max_n=6):
    done = False
    while not done:
        done = True
        n = random.randint(min_n, max_n)
        a = [['0' for i in range(n)] for j in range(n)]

        first_x = random.randint(0, n // 2)
        first_y = random.randint(0, n // 2)

        second_x = random.randint(first_x + 1, min(n, first_x + 1 + n // 2))
        second_y = random.randint(first_y + 1, min(n, first_y + 1 + n // 2))

        for X in range(first_x, second_x):
            for Y in range(first_y, second_y):
                a[X][Y] = 'r'

        size_x = second_x - first_x
        size_y = second_y - first_y

        generated = True
        current_id = 0
        count_cargos = 0
        while generated:
            if size_x <= 0 or size_y <= 0:
                break

            current_id += 1
            generated = False

            n_trials = 1000
            for _ in range(n_trials):
                success = True
                type_ = random.randint(0, 1)
                x_len, y_len = 1, 1
                if type_ == 0:
                    x_len = random.randint(1, size_x)
                else:
                    y_len = random.randint(1, size_y)

                if _ >= n_trials - 50:
                    x_len, y_len = 1, 1

                trial_x = random.randint(0, n - 1)
                trial_y = random.randint(0, n - 1)

                if trial_x + x_len >= n or trial_y + y_len >= n:
                    continue

                for X in range(trial_x, trial_x + x_len):
                    for Y in range(trial_y, trial_y + y_len):
                        if a[X][Y] != '0':
                            success = False
                    if not success:
                        break

                if success:
                    for X in range(trial_x, trial_x + x_len):
                        for Y in range(trial_y, trial_y + y_len):
                            a[X][Y] = str(current_id)

                    count_cargos += 1
                    generated = True
                    size_x -= y_len
                    size_y -= x_len

                    break
        if not count_cargos:
            done = False
            continue

        with open(file_name, 'w') as f:
            for line in a:
                for x in line:
                    f.write(f'{x} ')
                f.write('\n')

##Neural Network

In [ ]:
eps = 0.000001

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical

class ActorCritic(nn.Module):
    def __init__(self):
        super(ActorCritic, self).__init__()
        # 8 input coordinates
        self.affine = nn.Linear(8, 128)

        self.middle_layer = nn.Linear(128, 128)

        self.action_layer = nn.Linear(128, 4)
        self.value_layer = nn.Linear(128, 1)

        self.logprobs = []
        self.state_values = []
        self.rewards = []

    def forward(self, state):
        state = torch.from_numpy(state).float()

        state = self.affine(state)
        # state = torch.sigmoid(state)
        state = F.relu(state, inplace=False)
        state = self.middle_layer(state)
        # state = torch.sigmoid(state)
        state = F.relu(state, inplace=False)

        state_value = self.value_layer(state)

        act_l = self.action_layer(state)
        action_probs = F.softmax(act_l.add(eps), dim=0)
        action_distribution = Categorical(action_probs)
        action = action_distribution.sample()

        self.logprobs.append(action_distribution.log_prob(action))
        self.state_values.append(state_value)

        return action.item()

    def calculateLoss(self, gamma=0.99):
        # calculating discounted rewards:
        rewards = []
        dis_reward = 0
        for reward in self.rewards[::-1]:
            dis_reward = reward + gamma * dis_reward
            rewards.insert(0, dis_reward)
        rewards_length = len(rewards)

        # normalizing the rewards:
        rewards = torch.tensor(rewards)
        # Somehow normalizing rewards gives none in their value :(
        # rewards = (rewards - rewards.mean()) / (rewards.std() + eps)
        rewards.resize_(rewards_length, 1)

        loss = 0
        for logprob, value, reward in zip(self.logprobs, self.state_values, rewards):
            advantage = reward - value.item()
            action_loss = -logprob * advantage
            value_loss = F.smooth_l1_loss(value, reward)
            loss += (action_loss + value_loss)
        return loss

    def clearMemory(self):
        del self.logprobs[:]
        del self.state_values[:]
        del self.rewards[:]

##Train and test functions

In [ ]:
def test(policy, gamma, input_file, DEBUG=False, DEBUG2=False):
    policy.eval()

    world_shape, cargos_coordinates, cargos_corners, top_left, bottom_right = prepare_data(input_file)

    moves = []

    state = world_shape

    running_reward = 0
    total_cargos_length = 0
    total_number_of_actions = 0
    total_cargos_done = 0

    Total_reward = 0

    Total_length = 0
    for cargo_i in cargos_corners:
        Total_length += len(cargos_coordinates[cargo_i])

    counted_cargos = set()
    done_cargos = set()

    while running_reward != Total_length:
        for cargo_i in cargos_corners:

            if cargo_i not in counted_cargos:
                counted_cargos.add(cargo_i)
                total_cargos_length += len(cargos_coordinates[cargo_i])

            for t in range(1000):
                Current_state = []
                for coordinate_ in top_left:
                    Current_state.append(coordinate_)
                for coordinate_ in bottom_right:
                    Current_state.append(coordinate_)
                for coordinate_ in cargos_corners[cargo_i]:
                    Current_state.append(coordinate_)
                Current_state = np.array(Current_state).flatten()

                if DEBUG2:
                    print_world(world_shape, cargos_coordinates, cargos_corners, top_left, bottom_right)

                action = 0
                valid_action = False
                count_tries = 0
                while not valid_action and count_tries < 100:
                    count_tries += 1
                    valid_action = True

                    action = policy(Current_state)
                    change_x = (action % 2) * (1 if action in (1, 2) else -1)
                    change_y = (1 - action % 2) * (1 if action in (1, 2) else -1)
                    for coordinate in cargos_coordinates[cargo_i]:
                        valid_action &= (coordinate[0] + change_x >= 0 and coordinate[0] + change_x < len(world_shape))
                        valid_action &= (coordinate[1] + change_y >= 0 and coordinate[1] + change_y < len(world_shape[0]))

                if not valid_action:
                    break

                write_action = 'U'
                if change_x == 1:
                    write_action = 'D'
                elif change_y == 1:
                    write_action = 'R'
                elif change_y == -1:
                    write_action = 'L'

                moves.append((cargo_i, write_action))
                total_number_of_actions += 1

                # Changing the state after the move
                for coordinate in cargos_coordinates[cargo_i]:
                    coordinate[0] += change_x
                    coordinate[1] += change_y

                # Calculating the reward
                current_map = [[0 for y in range(len(x))] for x in world_shape]
                for Cargo in cargos_coordinates:
                    for coordinates in cargos_coordinates[Cargo]:
                        current_map[coordinates[0]][coordinates[1]] += 1

                reward = 0
                for I in range(len(current_map)):
                    for J in range(len(current_map[I])):
                        if current_map[I][J] > 1:
                            reward -= 1
                        if I >= top_left[0] and J >= top_left[1] and I <= bottom_right[0] and J <= bottom_right[1]:
                            reward += (1 if current_map[I][J] > 0 else 0)

                Total_reward += reward
                # Determining if we are done or not
                done = (reward == (len(cargos_coordinates[cargo_i]) + total_cargos_done))

                policy.rewards.append(reward)
                running_reward = reward

                if done:
                    if cargo_i not in done_cargos:
                        done_cargos.add(cargo_i)
                        total_cargos_done += len(cargos_coordinates[cargo_i])

                    policy.clearMemory()
                    break

    if DEBUG:
        print(f'Path length: {total_number_of_actions}')
    if DEBUG2:
        print_world(world_shape, cargos_coordinates, cargos_corners, top_left, bottom_right)

    return moves, Total_reward

In [ ]:
REWARD_VALUE = 100
Max_t = 1000

def train(policy, optimizer, gamma, input_file, DEBUG=False, DEBUG2=False):
    world_shape, cargos_coordinates, cargos_corners, top_left, bottom_right = prepare_data(input_file)

    state = world_shape

    running_reward = 0
    total_cargos_length = 0
    total_number_of_actions = 0
    total_cargos_done = 0

    Total_length = 0
    for cargo_i in cargos_corners:
        Total_length += len(cargos_coordinates[cargo_i])

    counted_cargos = set()
    done_cargos = set()
    while running_reward < Total_length * REWARD_VALUE:
        for cargo_i in cargos_corners:
            if cargo_i not in counted_cargos:
                counted_cargos.add(cargo_i)
                total_cargos_length += len(cargos_coordinates[cargo_i])

            for t in range(Max_t):
                Current_state = []
                for coordinate_ in top_left:
                    Current_state.append(coordinate_)
                for coordinate_ in bottom_right:
                    Current_state.append(coordinate_)
                for coordinate_ in cargos_corners[cargo_i]:
                    Current_state.append(coordinate_)
                Current_state = np.array(Current_state).flatten()

                if DEBUG2:
                    print_world(world_shape, cargos_coordinates, top_left, bottom_right)

                action = 0
                valid_action = False
                count_tries = 0
                while not valid_action and count_tries < 100:
                    count_tries += 1
                    valid_action = True

                    action = policy(Current_state)
                    change_x = (action % 2) * (1 if action in (1, 2) else -1)
                    change_y = (1 - action % 2) * (1 if action in (1, 2) else -1)
                    for coordinate in cargos_coordinates[cargo_i]:
                        valid_action &= (coordinate[0] + change_x >= 0 and coordinate[0] + change_x < len(world_shape))
                        valid_action &= (coordinate[1] + change_y >= 0 and coordinate[1] + change_y < len(world_shape[0]))

                if not valid_action:
                    break

                total_number_of_actions += 1

                prev_distance = 10 ** 9
                for coordinate in cargos_coordinates[cargo_i]:
                    prev_distance = min(prev_distance, abs(coordinate[0] - top_left[0]) + abs(coordinate[1] - top_left[1]))
                    prev_distance = min(prev_distance, abs(coordinate[0] - bottom_right[0]) + abs(coordinate[1] - bottom_right[1]))
                    prev_distance = min(prev_distance, abs(coordinate[0] - top_left[0]) + abs(coordinate[1] - bottom_right[1]))
                    prev_distance = min(prev_distance, abs(coordinate[0] - bottom_right[0]) + abs(coordinate[1] - top_left[1]))
                if coordinate[0] >= top_left[0] and coordinate[1] >= top_left[1] and coordinate[0] <= bottom_right[0] and coordinate[1] <= bottom_right[1]:
                    prev_distance = 0

                # Changing the state after the move
                for coordinate in cargos_coordinates[cargo_i]:
                    coordinate[0] += change_x
                    coordinate[1] += change_y

                cur_distance = 10 ** 9
                for coordinate in cargos_coordinates[cargo_i]:
                    cur_distance = min(cur_distance, abs(coordinate[0] - top_left[0]) + abs(coordinate[1] - top_left[1]))
                    cur_distance = min(cur_distance, abs(coordinate[0] - bottom_right[0]) + abs(coordinate[1] - bottom_right[1]))
                    cur_distance = min(cur_distance, abs(coordinate[0] - top_left[0]) + abs(coordinate[1] - bottom_right[1]))
                    cur_distance = min(cur_distance, abs(coordinate[0] - bottom_right[0]) + abs(coordinate[1] - top_left[1]))
                if coordinate[0] >= top_left[0] and coordinate[1] >= top_left[1] and coordinate[0] <= bottom_right[0] and coordinate[1] <= bottom_right[1]:
                    cur_distance = 0

                # Calculating the reward
                current_map = [[0 for y in range(len(x))] for x in world_shape]
                for Cargo in cargos_coordinates:
                    for coordinates in cargos_coordinates[Cargo]:
                        current_map[coordinates[0]][coordinates[1]] += 1

                reward = 0
                if cur_distance != 0:
                    reward += REWARD_VALUE / (cur_distance + 1)

                for I in range(len(current_map)):
                    for J in range(len(current_map[I])):
                        if current_map[I][J] > 1:
                            reward -= 1
                        if I >= top_left[0] and J >= top_left[1] and I <= bottom_right[0] and J <= bottom_right[1]:
                            reward += (1 * REWARD_VALUE if current_map[I][J] > 0 else 0)

                if cur_distance > prev_distance and cur_distance != 0:
                    reward -= 2 * REWARD_VALUE / (cur_distance + 1)

                # Determining if we are done or not
                done = (reward >= (len(cargos_coordinates[cargo_i]) + total_cargos_done) * REWARD_VALUE)

                policy.rewards.append(reward)
                running_reward = reward

                if done:
                    if cargo_i not in done_cargos:
                        done_cargos.add(cargo_i)
                        total_cargos_done += len(cargos_coordinates[cargo_i])

                    optimizer.zero_grad()
                    loss = policy.calculateLoss(gamma)
                    loss.backward()
                    optimizer.step()
                    policy.clearMemory()
                    break
                elif t + 1 == Max_t:
                    optimizer.zero_grad()
                    loss = policy.calculateLoss(gamma)
                    loss.backward()
                    optimizer.step()
                    policy.clearMemory()

    if DEBUG:
        print(f'Path length: {total_number_of_actions} reward: {running_reward}')
    if DEBUG2:
        print_world(world_shape, cargos_coordinates, top_left, bottom_right)
        print('---------------------------------')

##Testing the model

In [ ]:
import torch
import torch.optim as optim

gamma = 0.99
lr = 0.0001
betas = (0.9, 0.999)

policy = ActorCritic()
optimizer = optim.Adam(policy.parameters(), lr=lr, betas=betas)

In [ ]:
!rm -rf input
!mkdir input

In [ ]:
import torch

torch.autograd.set_detect_anomaly(True)
for i in range(1, 100):
    generate_random_map(f'input/input{1}.txt', min_n=2, max_n=6)
    print('-----------------------------------')
    print(f'Training iteration #{i}')
    train(policy=policy, optimizer=optimizer, gamma=gamma, input_file=f'input/input{1}.txt', DEBUG=True, DEBUG2=False)

-----------------------------------
Training iteration #1
Path length: 24 reward: 100
-----------------------------------
Training iteration #2
Path length: 6 reward: 100
-----------------------------------
Training iteration #3
Path length: 28 reward: 200
-----------------------------------
Training iteration #4
Path length: 1 reward: 200
-----------------------------------
Training iteration #5
Path length: 2 reward: 100
-----------------------------------
Training iteration #6
Path length: 10 reward: 400
-----------------------------------
Training iteration #7
Path length: 4 reward: 100
-----------------------------------
Training iteration #8
Path length: 5 reward: 200
-----------------------------------
Training iteration #9
Path length: 25 reward: 300
-----------------------------------
Training iteration #10
Path length: 11 reward: 100
-----------------------------------
Training iteration #11
Path length: 8 reward: 300
-----------------------------------
Training iteration #12

In [ ]:
torch.save(policy.state_dict(), 'policy.pt')

In [ ]:
solution, Total_reward = test(policy=policy, gamma=gamma, input_file=f'input1.txt', DEBUG=True, DEBUG2=False)

Path length: 46


In [ ]:
solution, Total_reward = test(policy=policy, gamma=gamma, input_file=f'input2.txt', DEBUG=True, DEBUG2=False)

Path length: 21


In [ ]:
solution, Total_reward = test(policy=policy, gamma=gamma, input_file=f'input3.txt', DEBUG=True, DEBUG2=False)

Path length: 88
